## T5 이용 보고서 요약

### T5 (Text-to-Text Transfer Transformer)
- 구글이 2019년에 발표한 모델
- 트랜스포머 구조 기반 모델로 인코더-디코더 구조
- 모든 NLP 작업을 text-to-text 형식으로 통일하여 처리
  - 입출력을 모두 텍스트로 다루는 것이 핵심 (text-to-text)
- 다양한 NLP 작업을 하나의 통일된 모델로 다룰 수 있게 설계
- 예
  - 감정 분석 :  '텍스트 분류' 작업으로 변환  
  - 내용 요약 :  '텍스트 요약' 작업으로 변환
- KoT5 : T5를 한국어 데이터로 학습시킨 모델 (paust/pko‑t5‑base모델)

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from transformers import TFT5ForConditionalGeneration, T5TokenizerFast
# 모델과 토크나이저 생성
model = TFT5ForConditionalGeneration.from_pretrained('paust/pko-t5-base', from_pt=True)
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')

C:\Anaconda3\envs\deepNLPenv\lib\site-packages\tf_keras\src\initializers\initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClass

### 어텐션 마스크 함수

In [9]:
import tensorflow as tf

# 입력 시퀀스에 대한 어텐션 마스크 생성
def create_attention_mask(input_ids):
    # input_ids와 패딩 토큰 ID를 비교하여 불리언 마스크 생성
    # 패딩이 아닌 토큰은 True,패딩 토큰은 False
    input_ids = tf.cast(input_ids, dtype=tf.int32)  # Ensure input_ids is int32
    boolean_mask =tf.not_equal(input_ids,tokenizer.pad_token_id)
    # 불리언 마스크를 float32타입으로 변환 (True->1.0, False-> 0.0)
    return tf.cast(boolean_mask, tf.float32)

In [ ]:
# 요약문 생성

In [11]:
def summarize(text, model, tokenizer, max_length=256):

     # 입력 텍스트 전처리
    input_text = 'summarize: ' + text
    #input_text = text
    #토큰화 및 입력 형식 준비
    inputs = tokenizer(input_text,return_tensors="tf", max_length=256,
                       truncation=True, padding='max_length')

    # attention_mask 생성 (create_attention_mask 함수 사용)
    attention_mask = create_attention_mask(inputs['input_ids'])

    # 요약문 생성
    summary_ids = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=attention_mask,
            max_length=max_length,
            #min_length=100, # 최소 길이 설정
            num_beams=6, # 값을 줄여 더 다양한 표현생성
            repetition_penalty=1.8,
            length_penalty=0.9, # 낮을수록 더 긴 요약을 유도
            no_repeat_ngram_size=2 #중복 단어 반복 방지
    )
    #토큰을 문자열로 변환
    summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)

    return summary

In [13]:
import pdfplumber
import pandas as pd

def pdf_to_dataframe(file_path):
    text_list = []  # 페이지별 텍스트 저장 리스트

    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text_list.append(page.extract_text())

    # 추출된 텍스트를 DataFrame으로 변환
    df = pd.DataFrame({'text': text_list})
    
    return df

    #return text_list

In [25]:
import re

def prelim_summary(text_list):
    
    text_str = " ".join(text_list)
    
    #각 섹션을 개별적으로 1차 요약 수행
    sections = [sec.strip() for sec in re.split(r'(?<!\d)\.(?!\d)\s+', text_str) if sec.strip() != '']
    print('섹션별 1차 요약:'+ str(sections))
    section_summaries = []
    for sec in sections:
        # 50자 미만 문장은 요약없이 원본 그대로 인용
        if len(sec) < 50:
            section_summaries.append(sec)
        else:
            summary_sec = summarize(sec, model, tokenizer)
            section_summaries.append(summary_sec)
    #개별 요약들을 최종 요약문으로 결합후 정리
    section_summaries = "\n".join(section_summaries)
    section_summaries = re.sub(r"[^가-힣\s]", "", section_summaries)
    section_summaries = re.sub(r"\s+", " ", section_summaries).strip() 
    return section_summaries

In [43]:
%%time
import pandas as pd
import glob
import os

# pdf 파일이 있는 폴더 경로 지정
folder_path = f'./data/경제전망보고서'
#folder_path = f'./data/통화신용정책보고서'
#folder_path = f'./data/통화정책방향결정문'
    
# 모든 pdf 파일 경로 가져오기
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))

# 빈 리스트 생성 (각 pdf 파일을 DataFrame으로 저장할 리스트)
dataframes_list = []

# 각 pdf 파일을 읽어서 리스트에 추가
for pdf_file in pdf_files:
    print('pdf_file:', pdf_file)
    # pdf 파일 읽기
    df_temp = pdf_to_dataframe(pdf_file)

    # 파일명 추출
    file_name = os.path.basename(pdf_file)
    
    # 텍스트 추출 및 요약 처리
    if 'text' in df_temp.columns:
        text_list = df_temp['text'].tolist()
        
        # prelim_summary 함수가 문자열을 반환
        summary_text = prelim_summary(text_list)
        
        # 요약 텍스트와 파일명을 딕셔너리로 저장
        summary_data = {
            'summary': summary_text,
            'source_file': file_name
        }
        
        # 리스트에 추가
        dataframes_list.append(summary_data)
        
    # 모든 DataFrame을 하나로 결합
    if dataframes_list:
        #df = pd.concat(dataframes_list, ignore_index=True)
        df = pd.DataFrame(dataframes_list)
        # 파일 저장 경로 수정
        output_path = os.path.join(folder_path, 'summary_reports.csv')
        
        # 디렉토리가 존재하는지 확인하고 없으면 생성
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # CSV 파일로 저장
        df.to_csv(output_path, encoding='utf-8')

pdf_file: ./data/경제전망보고서\경제전망 요약_202302.pdf
섹션별 1차 요약:['요 약\n\uf000 최근 국내외 여건변화 등을 감안할 때, 경제성장률은 금년과\n내년중 각각 1.6%, 2.4% 수준을 나타낼 전망\nㅇ 향후 국내경제는 글로벌 경기둔화, 금리상승 등의 영향으로\n상반기중 부진한 성장흐름을 이어갈 것으로 예상되며\n하반기 이후에는 중국 및 IT경기 회복 등으로 점차 나아질 것\n으로 예상되지만 전망의 불확실성은 높은 상황\n▪민간소비는 실질구매력 둔화, 원리금 상환부담 증대 등으로\n회복세가 완만할 것으로 예상\n▪설비투자는 글로벌 경기둔화와 금융비용 증대 등의 영향으로\n부진할 전망\n▪건설투자는 주택경기 둔화, SOC예산 감소 등이 하방요인으로\n작용\n▪상품수출은 글로벌 경기부진으로 당분간 둔화흐름이 이어지다가\n하반기 이후 중국·IT경기 회복 등에 힘입어 점차 개선될 전망\n\uf000 취업자수는 금년과 내년중 각각 13만명, 15만명 증가할 전망\nㅇ 지난해 리오프닝에 따른 효과가 축소되고 경기둔화의 영향이\n나타나면서 소폭 증가에 그칠 전망 \uf000 소비자물가 상승률은 금년과 내년중 각각 3.5%, 2.6%를 나타낼 전망\nㅇ 올해는 국제유가가 작년보다 낮아지고 경기가 둔화되는 등 공급\n및 수요측 물가압력이 모두 약화되면서 소비자물가 상승률이\n지난해(5.1%)보다 상당폭 낮아질 것으로 예상\n\uf000 경상수지 흑자규모는 금년과 내년중 각각 260억달러, 480억달러를\n기록할 전망\nㅇ GDP 대비 경상수지 흑자 비율은 금년 및 내년중 각각 1% 중반,\n2% 중반 수준으로 예상\n경제 전망\n(%)\n2022 2023e) 2024e)\n상반 하반 연간 상반 하반 연간 연간\n▪GDP1) 3.0 2.2 2.6 1.1 2.0 1.6 2.4\n민간소비 4.1 4.6 4.4 3.3 1.3 2.3 2.4\n설비투자 -6.4 5.4 -0.7 3.2 -8.9 -3.1 3.6\n지식재산생산물투자 4.6 5.0 4.8 